In [8]:
from gensim.models import FastText
import pandas as pd
import numpy as np
import itertools
import html
import pickle
import regex as re

from sklearn.metrics import confusion_matrix
from nltk.tokenize import TweetTokenizer
from collections import Counter
from emoji import UNICODE_EMOJI
import emoji
import matplotlib.pyplot as plt
%matplotlib inline

c:\users\thora\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [34]:
def preprocess_dataframe(input_path, output_path):
    df = pd.read_csv(input_path, sep='\t', encoding='utf8')
    text = df['text']
    labels = df['label']
    tk_texts = tokenize_texts(text)
    new_texts = []
    for t in tk_texts:
        text = " ".join(t)
        new_texts.append(text)
    df_preprocessed = pd.DataFrame(columns=['label', 'text'])
    for i, text in enumerate(new_texts):
        df_preprocessed = df_preprocessed.append({'label': labels[i], 'text':text}, ignore_index=True)
    df_preprocessed.to_csv(output_path, sep='\t', encoding='utf8')

In [26]:
def tokenize_texts(corpus, stopword_path = None):
    """
    Tokenizes a list of texts. ...!
    """
    corpus = convert_html_emojis(corpus)
 
    tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True) 
    
    token_doc_list = []
    token_list = []
   
    for text in corpus:

        tokens = tknzr.tokenize(text)
        tokens2 = []
        for token in tokens:
            token = re.sub(r'([1234567890!"#$%&()*+,./:@;?[\]^`{|}_~\t\n])', '', token)
            if token != '' and token != 'charlie' and token !='yankees':
                tokens2.append(token)

        tmp_tokens = []
        for i, token in enumerate(tokens2):
            if token in UNICODE_EMOJI:
               
                emoji_one = emoji.demojize(token)
                emoji_one = emoji_one[1:-1]
                emoji_all = re.split('_', emoji_one)

                tokens[i] = emoji_all
                token_list.extend(emoji_all)
                tmp_tokens.extend(emoji_all)
            elif token == "i'd":
                token_list.extend(['i', 'would'])
                tmp_tokens.extend(['i', 'would'])
            else: 
                token_list.append(token)
                tmp_tokens.append(token)

            
        token_doc_list.append(tmp_tokens)     

    return token_doc_list

In [19]:
def convert_html_emojis(corpus):
    new_corpus = []
    for text in corpus:
        text = html.unescape(text)
        new_corpus.append(text)
    return new_corpus

In [20]:
train_path = 'train_data.csv'
dev_path = 'dev_data.csv'
test_path = 'test_data.csv'

In [35]:
preprocess_dataframe(train_path, 'train_preprocessed.tsv')

In [36]:
preprocess_dataframe(test_path, 'test_preprocessed.tsv')

In [38]:
preprocess_dataframe(dev_path, 'dev_preprocessed.tsv')

In [41]:
df_wtf = pd.read_csv('dev_preprocessed.tsv', sep='\t', index_col=0, engine='python', encoding='utf8')

In [42]:
df_wtf

,label,text
0,0,add me back you twat
1,2,ruffled ntac eileen dahlia - beautiful color c...
2,1,i know bro nigga seen a ' african playing guar...
3,0,w left in the game are you retarded
4,0,none of us went to these prestigious establish...
5,1,i called at least times to make my hair appoin...
6,1,i also like flip a bitch which is gabbys quali...
7,1,why is that most of the niggas bitches i use t...
8,0,eee u sick fuck what do u even do oo politics ...
9,0,you i was like you sneaky shit i saw him befor...


In [43]:
lines = df_wtf.values.tolist()

In [44]:
lines

[[0, 'add me back you twat'],
 [2,
  'ruffled ntac eileen dahlia - beautiful color combination of pink orange yellow white a coll httptcoHdYEBvnZB'],
 [1,
  "i know bro nigga seen a ' african playing guard and was handling the hoe"],
 [0, 'w left in the game are you retarded'],
 [0,
  "none of us went to these prestigious establishments so we're clearly not as superior as these fucking retards"],
 [1,
  "i called at least times to make my hair appointment and these bitches don't pick - - guess i have to stop in to make an appointment grrr"],
 [1, 'i also like flip a bitch which is gabbys qualitywords'],
 [1,
  'why is that most of the niggas bitches i use to fw like to sub me face with tears of joy loudly crying face'],
 [0, 'eee u sick fuck what do u even do oo politics you fuckin odd cunt'],
 [0,
  'you i was like you sneaky shit i saw him before i saw your dm because my phone is retarded'],
 [2, 'i cannot get this goofy grin off my face'],
 [1,
  "rt my body thought we'd get an extr

In [53]:
import six
def convert_to_unicode(text):
  """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
  if six.PY3:
    if isinstance(text, str):
      return text
    elif isinstance(text, bytes):
      return text.decode("utf-8", "ignore")
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  elif six.PY2:
    if isinstance(text, str):
      return text.decode("utf-8", "ignore")
    elif isinstance(text, unicode):
      return text
    else:
      raise ValueError("Unsupported string type: %s" % (type(text)))
  else:
    raise ValueError("Not running on Python2 or Python 3?")

In [55]:
for (i, line) in enumerate(lines):
    text_a = convert_to_unicode(str(line[1]))
    label = convert_to_unicode(str(line[0]))
    print(text_a)
    print(label)
    print('\n')

add me back you twat
0


ruffled ntac eileen dahlia - beautiful color combination of pink orange yellow white a coll httptcoHdYEBvnZB
2


i know bro nigga seen a ' african playing guard and was handling the hoe
1


w left in the game are you retarded
0


none of us went to these prestigious establishments so we're clearly not as superior as these fucking retards
0


i called at least times to make my hair appointment and these bitches don't pick - - guess i have to stop in to make an appointment grrr
1


i also like flip a bitch which is gabbys qualitywords
1


why is that most of the niggas bitches i use to fw like to sub me face with tears of joy loudly crying face
1


eee u sick fuck what do u even do oo politics you fuckin odd cunt
0


you i was like you sneaky shit i saw him before i saw your dm because my phone is retarded
0


i cannot get this goofy grin off my face
2


rt my body thought we'd get an extra hr sleep but my brain decided to wake up hrs earlier my brains a bitch
1


still ken fuck all u cunt
0


why are you so butthurt faggot
0


oooh bitch hv u deactivate ur cunt self smirking face smirking face smirking face
0


one of us one of us
2


sweet milf in white lingerie has her hose ripped and her pussy pounded httptcoCMacswjNA
1


“ shorty name is masika and y'all surprised she a hoe ” face with tears of joy
1


imma tie that bitch up in the basement and donkey-fuck her ass for two weeks straight hahahahahahahahaha centmovie beforeiselfdestruct
1


rt she literally just put the whole trash bag on lmfao httptcoYiERskYrfc
2


\ nblake is a retard for letting you go to keep him
0


always okay to bring your ho to the barbershop though they usually cool af
1


rt thanks carolina fans the flipped birds few bottles to the head sticks n stones may break my bones but aluminum zimas will …
2


only leo
2


“ im drunk i'm bout to bitch ” need a ride
1


rt tell me again how citizen kane is the best movie of all time then go watch caddyshack you pretentious fil

“ adam jones homers now lead - ” guy is a monsta
2


thanks for coming with me last night because it's ghetto and no one else could of handled it summer
2


dear cunt you keep your muslim loving metrosexual of a leader i have no doubt islam will be conquering your pitiful nation
0


rt it really feels like the dm can be the hand-on-the-knee of social communication httptcomfselzfe stopwad …
2


someone posted this on facebook in response to that woman tweeting at me earlier today httptcobywkrelyo
2


hey you have a retarded leftover from anti-gamergate stalking you lol
0


retweet to spam this faggot rt dick going backhand index pointing up
1


awww
2


as long as she realizes she \ ' s not gonna look as pretty as she usually works this character is kind of a mess
2


ur the one dodging retard lmfao send any amount httpstcoDyGkQKhOd
0


shut the fuck up dummy jimmy neutron retard your ass video is shit and will never be good
0


mt browngal gouache monkey tiki httptcokFhjlWhbs
2


rt li


no
2


horrible twat hope you wake up and you find you dead mums head in the oven
0


chippah i hopes you ain't be hangin wit dis wigger boy
1


rt bitches be quick to say let me pull out my phone face with tears of joy face with tears of joy face with tears of joy
1


deadass wanted the sauce till shit got real then niggas wanna back up ooh iight then smh pussy
1


“ fuck me then bitch ” shut up and roll threw
1


thanks bitch
1


face with tears of joy face with tears of joy face with tears of joy i will beast your twat if you carry on middle finger 🏼
0


you just felt like saying nigger or you was trying to make a point
0


lesson don't pass out at before like a lil bitch
1


yeah tht a bitch we ain't gay brah chill
1


wtf you a simple sorry cunt no beter shit to say
0


icanhashtaghoweveriwant bitch
1


nigger living jew you should kys
0


not exactly the part i was relaying but p pp
2


rt i was about to be like r u queer but c'mon jen do u even need to ask
1


\ n-shrimp fest \



rt muslim jihadi fighters book tickets on cruise ships to syria and iraq httptcoauFmcZY
2


lmao bitch is dumb slap her up
1


bitch you fucking told me to sit in that seat and yo dumbass come again saying why you sitting there bitch votesos
1


dw i can be a retard for u again
0


i can make a dike say she leavin that lady i can make a lady say she leavin that pussy ass nigga that gave her that baby
1


follow me you nigger
0


lol time to look at a new social group and write a blocklist i guess
2


rt wtf ling ling did you at least calculate the circumference when you threw that ass in a circle httptcovLGMVAT
1


i have a sunburn
2


that nigga spun around in the pussy
1


ipad air or ipad mini
2


rt mayonnaise colored benz i push miracle whips httptcoBmXuyPehH
2


always except when we played with that annoying ass biazte kid you always use to tag that retard
0


rt how shy people flirt learn tips from the best httptcoPvuHnHaT
2


on my way to twitter be right there
2


oh no so 

1


we don't love them hoes
1


rt most of these niggas go broke for these hoes
0


hey who invited you faggot
0


hidey hidey hoe httptcofPSgBhk
1


morning you big cunt btw ily
0


praise thy lord for this season of beanies that is upon us
2


the fuck u blank tweeting for faggot
0


broken boats lmao you probably live on a broken boat you fucking autistic cunt
0


hba trash
2


that is behind the counter
2


boy you think im joking look at the fucking price you retard
0


rt fat bitches dont like liftin up they arms in the summer all that dampness cuh
1


rt approach yo bitch like httptcoUHMIMEpsw
1


rt the new nature area replacing indian boundary park's former zoo is expected to attract a variety of migratory birds http…
2


rt please report this httpstcoizmfmcqqc abuse of api and privacy hack …
2


dat bitch had noo cheeks
1


why would you do that p these lectures are about freebsd
2


tempted to finish every sentence i say today with yo or bitch breakingbad
1


you gotta be li

In [49]:
l = list(df_wtf['label'])

In [50]:
from collections import Counter
Counter(l)

Counter({0: 1000, 2: 1000, 1: 1000})